In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io

train_identification = pd.read_csv('dm2020-hw2-nthu/data_identification.csv')
train_emotion = pd.read_csv('dm2020-hw2-nthu/emotion.csv')
sample_submit = pd.read_csv('dm2020-hw2-nthu/sampleSubmission.csv')


/home/neaf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/neaf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/neaf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/neaf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
json_feature = pd.read_json('dm2020-hw2-nthu/tweets_DM.json', lines=True)

In [3]:
data = pd.merge(train_identification, train_emotion, on=["tweet_id"])
data.head()

tweet_id identification       emotion
0  0x29e452          train           joy
1  0x2b3819          train           joy
2  0x2a2acc          train         trust
3  0x2a8830          train           joy
4  0x20b21d          train  anticipation

In [4]:
identification_train = data['identification'] == 'train'
identification_test = train_identification['identification'] == 'test'

train_X = data.loc[identification_train]
test_X = train_identification.loc[identification_test]

In [5]:
test_X['emotion'] = np.nan
df = pd.concat([train_X, test_X])

json_feature['hashtags'] = np.nan; json_feature['text'] = np.nan
json_extend = pd.io.json.json_normalize(json_feature._source)
json_extend['tweet.text'].isna().sum()
json_feature['hashtags'] = json_extend['tweet.hashtags']
json_feature['text'] = json_extend['tweet.text']
json_feature['tweet_id'] = json_extend['tweet.tweet_id']

df = pd.merge(df, json_feature, on=["tweet_id"])

/home/neaf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/neaf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


In [6]:
df.shape

(1867535, 10)

In [7]:
df_train = df.loc[df['identification'] == 'train']
df_test = df.loc[df['identification'] == 'test']

#X = data['text'].values
#y = data['emotion'].values

#X_train, X_test, y_train, y_test = train_test_split(X_df['text'], X_df['emotion'], test_size=0.33)

In [8]:
device = torch.device("cuda")
#device = torch.device("cpu")

In [9]:
df_train['emotion'].unique()

array(['joy', 'trust', 'anticipation', 'sadness', 'disgust', 'fear',
       'surprise', 'anger'], dtype=object)

In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_train['emotion_enc'] = labelencoder.fit_transform(df_train['emotion'])

/home/neaf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
df_train[['emotion','emotion_enc']].drop_duplicates(keep='first')

emotion  emotion_enc
0            joy            4
2          trust            7
4   anticipation            1
8        sadness            5
13       disgust            2
45          fear            3
58      surprise            6
68         anger            0

In [12]:
df_train.rename(columns={'emotion':'label_desc'},inplace=True)
df_train.rename(columns={'emotion_enc':'emotion'},inplace=True)


/home/neaf/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
sentences = df_train.text.values
#check distribution of data based on labels
print("Distribution of data based on labels: ",df_train.emotion.value_counts())

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]
emotion = df_train.emotion.values

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

Distribution of data based on labels:  4    516017
1    248935
7    205478
5    193437
2    139101
3     63999
6     48729
0     39867
Name: emotion, dtype: int64


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Actual sentence before tokenization:  @KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well done team 🌟 <LH> of every one of you.
Encoded Input from dataset:  [101, 1030, 4268, 7666, 1030, 27263, 2226, 1035, 4647, 2232, 1030, 7910, 9818, 5358, 5244, 1030, 1038, 16526, 2232, 15853, 2015, 2092, 2589, 2136, 100, 1026, 1048, 2232, 1028, 1997, 2296, 2028, 1997, 2017, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [20]:
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,
                                                                                 emotion,
                                                                                 random_state=41,
                                                                                 test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,
                                                    input_ids,
                                                    random_state=41,
                                                    test_size=0.1)

In [21]:
# convert all our data into torch tensors, required data type for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [22]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)
model = model.to(device)
# Parameters:
lr = 2e-5
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for _ in tnrange(1,epochs+1,desc='Epoch'):
  print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    
    # Backward pass
    loss.backward()
    
    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()
    
    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    
    df_metrics=pd.DataFrame({'Epoch':epochs,'Actual_class':labels_flat,'Predicted_class':pred_flat})
    
    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)
    
    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')

/home/neaf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  if __name__ == '__main__':


<====================== Epoch 1 ======================>


In [ ]:
# Result
#train_X = feature.fit_transform(X_df['text'])
X_submit_preprocessed = np.array([text_preprocessing(text) for text in Xsubmit['text']])

X_submit = tf_idf.transform(X_test_preprocessed)



In [122]:
pred_y = model.predict(X_submit)

In [124]:
Xsubmit['emotion'] = pred_y

/home/neaf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [125]:
Xsubmit.head()

tweet_id identification  emotion  _score          _index  \
1455563  0x28cc61           test    trust     107  hashtag_tweets   
1455564  0x2db41f           test  sadness     728  hashtag_tweets   
1455565  0x2466f6           test      joy     491  hashtag_tweets   
1455566  0x23f9e9           test    trust      28  hashtag_tweets   
1455567  0x1fb4e1           test      joy     925  hashtag_tweets   

                                                   _source  \
1455563  {'tweet': {'hashtags': [], 'tweet_id': '0x28cc...   
1455564  {'tweet': {'hashtags': [], 'tweet_id': '0x2db4...   
1455565  {'tweet': {'hashtags': ['womendrivers'], 'twee...   
1455566  {'tweet': {'hashtags': ['robbingmembers'], 'tw...   
1455567  {'tweet': {'hashtags': [], 'tweet_id': '0x1fb4...   

                  _crawldate   _type          hashtags  \
1455563  2017-01-17 14:13:32  tweets                []   
1455564  2015-10-17 06:46:20  tweets                []   
1455565  2016-12-19 03:50:27  tweets    [womendrivers]   
1455566  2017-04-09 19:32:19  tweets  [robbingmembers]   
1455567  2016-01-15 11:59:31  tweets                []   

                                                      text  
1455563  @Habbo I've seen two separate colours of the e...  
1455564  @FoxNews @KellyannePolls No serious self respe...  
1455565  Looking for a new car, and it says 1 lady owne...  
1455566  @cineworld “only the brave” just out and fount...  
1455567  Felt like total dog 💩 going into open gym and ...

In [126]:
submit = pd.DataFrame()
submit['id'] = Xsubmit['tweet_id']; submit['emotion'] = Xsubmit['emotion']


In [127]:
submit.to_csv('submission.csv', index=False)

In [128]:
submit

id  emotion
1455563  0x28cc61    trust
1455564  0x2db41f  sadness
1455565  0x2466f6      joy
1455566  0x23f9e9    trust
1455567  0x1fb4e1      joy
...           ...      ...
1867530  0x2c4dc2      joy
1867531  0x31be7c    trust
1867532  0x1ca58e    trust
1867533  0x35c8ba      joy
1867534  0x1d941b      joy

[411972 rows x 2 columns]